<a href="https://colab.research.google.com/github/pedrofernandss/reconhecimento_caes/blob/main/01_definicao_baseline_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install pyngrok

In [72]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from pyngrok import ngrok
from google.colab import userdata
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [73]:
get_ipython().system_raw("mlflow ui --host 0.0.0.0 --port 5000 &")
ngrok_auth_token = userdata.get('ngrok_token')

ngrok.set_auth_token(ngrok_auth_token)
ngrok.kill()
public_url = ngrok.connect(addr="5000", proto="http", bind_tls=True)

mlflow.set_tracking_uri("http://localhost:5000")

print(f"MLflow acessível em: {public_url}")

MLflow acessível em: NgrokTunnel: "https://f57e5527089c.ngrok-free.app" -> "http://localhost:5000"


In [74]:
data_directory = '/content/drive/MyDrive/reconhecimento_caes/dados_organizados/'

In [75]:
train_set = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels='inferred',
    validation_split = 0.2,
    subset = "training",
    seed=123,
    image_size = (224, 224),
    batch_size = 32,

)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels='inferred',
    validation_split = 0.2,
    subset = "validation",
    seed=123,
    image_size = (224, 224),
    batch_size = 32
)

Found 115 files belonging to 25 classes.
Using 92 files for training.
Found 115 files belonging to 25 classes.
Using 23 files for validation.


In [76]:
def preprocessing_pipeline(image: tf.Tensor, label: tf.Tensor):
  resized_image = tf.image.resize(image, [224, 224])
  normalized_image = tf.keras.applications.resnet50.preprocess_input(resized_image)

  return normalized_image, label

In [77]:
train_dataset = train_set.map(preprocessing_pipeline)
validation_dataset = validation_set.map(preprocessing_pipeline)

In [78]:
base_model = tf.keras.applications.ResNet50(
    include_top = False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

base_model.trainable = False

In [79]:
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(units=25, activation='softmax')
])

In [80]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [82]:
mlflow.set_experiment("Artigo - Reconhecimento de Cães")
mlflow.tensorflow.autolog()

with mlflow.start_run(run_name="Baseline de Classificação com Resnet50"):
    history = model.fit(
        train_dataset,
        epochs=10,
        validation_data=validation_dataset
    )

2025/07/14 21:34:23 INFO mlflow.tracking.fluent: Experiment with name 'Artigo - Reconhecimento de Cães' does not exist. Creating a new experiment.


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.0269 - loss: 4.0426

3/3 ━━━━━━━━━━━━━━━━━━━━ 29s 7s/step - accuracy: 0.0365 - loss: 4.0021 - val_accuracy: 0.0870 - val_loss: 3.0552
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1943 - loss: 2.5950

3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 6s/step - accuracy: 0.2055 - loss: 2.5923 - val_accuracy: 0.1739 - val_loss: 2.7934
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3841 - loss: 2.0771

3/3 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - accuracy: 0.3886 - loss: 2.0485 - val_accuracy: 0.3913 - val_loss: 2.4390
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6445 - loss: 1.3495

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step - accuracy: 0.9828 - loss: 0.2237 - val_accuracy: 0.6522 - val_loss: 1.4410
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 1.0000 - loss: 0.1889

3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step - accuracy: 1.0000 - loss: 0.1862 - val_accuracy: 0.6522 - val_loss: 1.3699
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 1.0000 - loss: 0.1444

3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 6s/step - accuracy: 1.0000 - loss: 0.1407 - val_accuracy: 0.6522 - val_loss: 1.3307
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


2025/07/14 21:39:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Baseline de Classificação com Resnet50 at: http://localhost:5000/#/experiments/254088716510997524/runs/0e069447f0944b33ab16f7fce6746612
🧪 View experiment at: http://localhost:5000/#/experiments/254088716510997524


In [84]:
print("Avaliando o modelo diretamente no conjunto de validação...")
results = model.evaluate(validation_dataset)
print("\nResultados da Avaliação:")
print(f"Perda de Validação (Loss): {results[0]}")
print(f"Acurácia de Validação (Accuracy): {results[1]}")

Avaliando o modelo diretamente no conjunto de validação...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6522 - loss: 1.3307

Resultados da Avaliação:
Perda de Validação (Loss): 1.3306883573532104
Acurácia de Validação (Accuracy): 0.6521739363670349
